In [ ]:
from fastapi import FastAPI, File, UploadFile
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN  # For MTCNN
from tensorflow.keras.models import load_model  # For SSD

# Replace with paths to pre-trained models
haar_cascade_path = "path/to/haarcascade_frontalface_default.xml"
ssd_model_path = "path/to/ssd_mobilenet_v2_face.h5"  # Example SSD model

# Load models
face_cascade = cv2.CascadeClassifier(haar_cascade_path)
ssd_model = load_model(ssd_model_path)

app = FastAPI()

def detect_faces_haar(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    return faces

def detect_faces_mtcnn(image):
    detector = MTCNN()
    faces = detector.detect_faces(image)
    return faces

def detect_faces_ssd(image):
    # Preprocess image for SSD (example assuming RGB format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.expand_dims(image, axis=0)
    # Perform prediction with SSD model
    boxes = ssd_model.predict(image)[0]
    # Extract relevant information from detections (adjust based on model output)
    faces = []
    for box in boxes:
        if box[0] > 0.5:  # Adjust threshold based on model
            x_min, y_min, x_max, y_max = int(box[1] * image.shape[1]), int(box[2] * image.shape[0]), int(box[3] * image.shape[1]), int(box[4] * image.shape[0])
            faces.append([x_min, y_min, x_max, y_max])
    return faces

@app.post("/detect")
async def detect_image(algorithm: str = "haar", image: UploadFile = File(...)):
    # Read image content
    image_bytes = await image.read()
    # Decode image
    image = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)

    # Perform face detection based on chosen algorithm
    if algorithm == "haar":
        faces = detect_faces_haar(image.copy())
    elif algorithm == "mtcnn":
        faces = detect_faces_mtcnn(image.copy())
    elif algorithm == "ssd":
        faces = detect_faces_ssd(image.copy())
    else:
        return {"error": "Invalid algorithm selection"}

    # Prepare results (adjust based on your needs)
    results = []
    for face in faces:
        x_min, y_min, x_max, y_max = face
        results.append({"bbox": [x_min, y_min, x_max, y_max]})

    return results

if __name__ == "__main__":
    import uvicorn
    uvicorn.run("main:app", host="0.0.0
